In [1]:
import numpy as np
from PIL import Image
import os
import csv
import string
import pickle
import matplotlib.pyplot as plt
from pickle import dump,load
# from pickle import load
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
#from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.xception import Xception,preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
#from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.resnet import ResNet50,preprocess_input 
# from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# from tensorflow.keras.preprocessing.image import load_img,
# from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer #for text tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.layers.merge import add
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense,LSTM, Embedding, Dropout,add,Conv1D,MaxPooling1D,Flatten

In [3]:
def img_preprocessing(model,size):
    images = []
    imagesid = []
    # Import images
    #/drive/MyDrive/ColabNotebooks
    dir = os.path.join('./data/Images')
    for img_file in os.listdir(dir):
        imgpath = dir + '/' + img_file
        # Load image to pixel map (224x244 for VGG16 input format)
        img_raw = load_img(imgpath, target_size=size)
        # Convert to numpy array
        image = img_to_array(img_raw)
        # Perform VGG16 preprocessing
        if (model==model_VGG or model==model_RES):
   
          image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            #Perform VGG16 preprocessing
          image = preprocess_input(image)

        if(model==model_INC or model==model_XCE):
            image = np.expand_dims(image, axis=0)
            image = image/127.5
            image = image - 1.0
        # Add image to output
        imgfeature = model.predict(image, verbose=0)
        # Add image to output
        imageid = img_file.split('.')[0]
        images.append(imgfeature) 
        imagesid.append(imageid)
    return images,imagesid

def text_preprocessing():
    captions =[]
    img_dict = {}
    with open(os.path.join( './data/captions.txt'), 'r') as f:
        next(f)
        captions_doc = f.read() 
    # process lines
    for line in (captions_doc.split('\n')):

        tokens = line.split(',')
        if len(line) < 2:
            continue
        imgid, caption = tokens[0], tokens[1:]
       
        imgid = imgid.split('.')[0]
      
        caption = " ".join(caption)
        caption = caption.lower()
        caption = caption.replace('[^A-Za-z]', '')
        caption = caption.replace('\s+', ' ')
        caption =" ".join([x for x in caption.split() if len(x)>1]) +' sequence_end'
        captions.append(caption)              

        if imgid not in img_dict:
            img_dict[imgid] = []
    
        img_dict[imgid].append(caption)
    
    return img_dict,captions

In [ ]:
model_VGG = VGG16()

model_INC = InceptionV3()
model_XCE = Xception()
model_RES = ResNet50()

# model_VGG = VGG16()
model_VGG = Model(inputs=model_VGG.inputs, outputs=model_VGG.layers[-2].output)
model_RES = Model(inputs=model_RES.inputs, outputs=model_RES.layers[-2].output)
#features_VGG = {}
i,j=img_preprocessing(model_VGG,(224, 224))
features_VGG = dict(zip(j, i))

# model_INC = InceptionV3()
# #features_INC = {}
# i,j=img_preprocessing(model_INC,(299, 299))
# features_INC = dict(zip(j, i))

# model_XCE = Xception()
# #features_XCE = {}
# i,j=img_preprocessing(model_XCE,(299, 299))
# features_XCE = dict(zip(j, i))

# model_RES = ResNet50()
# features_RES = {}
# i,j=img_preprocessing(model_RES,(224, 224))
# features_RES = dict(zip(j, i))

In [ ]:
with open('./csvdata/VGG.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write the data
    writer.writerow(features_VGG)

# with open('./csvdata/INC.csv', 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)
#     # write the data
#     writer.writerow(features_INC)

# with open('./csvdata/XCE.csv', 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)
#     # write the data
#     writer.writerow(features_XCE)

# with open('./csvdata/RES.csv', 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)
#    # write the data
#     writer.writerow(features_RES)

In [ ]:
img_data,caption_data=text_preprocessing()
with open('./csvdata/caption_map.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write the data
    writer.writerow(img_data)

with open('./csvdata/captions.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write the data
    writer.writerow(caption_data)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(caption_data)
vocab_size = len(tokenizer.word_index) + 1


max_length = max(len(caption.split()) for caption in caption_data)

In [ ]:
def prepare_data(image_keys,features):
    
    x1, x2, y = [], [], []

    for image in image_keys:
        captions = img_data[image]
        for caption in captions:
          seq = tokenizer.texts_to_sequences([caption])[0]
          length = len(seq)

          for i in range(1, length):
              x2_seq, y_seq = seq[:i] , seq[i]  
              x2_seq = pad_sequences([x2_seq], maxlen = max_length)[0]       
              y_seq = to_categorical([y_seq], num_classes = vocab_size)[0]

              x1.append( features[image][0] )
              x2.append(x2_seq)
              y.append(y_seq)
               
    return np.array(x1), np.array(x2), np.array(y)

In [ ]:
image_ids = list(img_data.keys())
split = int(len(image_ids) * 0.90)
train = image_ids[:split]
test = image_ids[split:]

inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)


decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)



model_VGG = Model(inputs=[inputs1, inputs2], outputs=outputs)
model_VGG.compile(loss=['categorical_crossentropy'], optimizer='adam',metrics=['accuracy','sparse_categorical_accuracy'],loss_weights=[0.001])

In [ ]:
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# ModelCheckpoint
# monitor validation loss and save the best model weights
checkpoints = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join('./models'),
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False
)



# tensorboard = keras.callbacks.TensorBoard(
#     log_dir=os.path.join('./logs')
# )

# ReduceLROnPlateau
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    min_lr=1e-5,
    verbose=1
)


In [ ]:

train_x1, train_x2, train_y = prepare_data(train,features_VGG)
validate_x1, validate_x2, validate_y = prepare_data(test,features_VGG)
model_VGG.fit([train_x1, train_x2],  train_y,  verbose = 1, epochs = 5, callbacks = [reduce_lr, early_stop, checkpoints],validation_data=([validate_x1, validate_x2], validate_y))



In [ ]:
def predict_caption(model,image):
   
    in_text = ''  
    for i in range(max_length):       
        sequence = tokenizer.texts_to_sequences([in_text])[0]       
        sequence = pad_sequences([sequence], max_length)       
        yhat = model.predict([image, sequence], verbose=0)       
        yhat = np.argmax(yhat)
      
        text_word=''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                text_word=word
            else:
                text_word=None     


        if text_word is None:
            break
       
        if text_word == 'sequence_end':
            break
        in_text += " " + text_word
    return in_text

In [ ]:
def imagecaption(image_name):

    # image_name = "1001773457_577c3a7d70.jpg"
    image_id = image_name.split('.')[0]
    img_path = os.path.join("./drive/MyDrive/ColabNotebooks/data/Images", image_name)
    image = Image.open(img_path)
    
    y_pred = predict_caption(model_VGG, features_VGG[image_id])
    plt.imshow(image)
    print(print('Caption Generated: '),y_pred)



imagecaption("1001773457_577c3a7d70.jpg")